In [1]:
import os
import sys
import re
import argparse
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
organism_name = "Chlamydia psittaci"
#organism_name = "Vibrio campbellii"
lst = organism_name.split()
lst = re.split(" |_", organism_name)
if len(lst[-1]) < 10:
    last_letter = len(lst[-1])
else:
    last_letter = 9
folder_name = f"{lst[0][0]}_{lst[-1][:last_letter]}"

### Orto rows with singletons, but without paralog-containing rows

In [3]:
# Uploading and modificating orto rows tables
orto_rows = pd.read_csv(f"../{folder_name}/data/{folder_name}.proteinortho.tsv", sep="\t") # Uploading dataframe with orto rows
orto_rows = orto_rows.rename(columns = {"# Species": "Species"})
orto_rows["ortologus_row"] = orto_rows.index + 1 # Creating ortorows numbers

# Uploading first csv-table and creating a new column in it
orto_rows_list = orto_rows.index
df1 = pd.read_csv(f"../{folder_name}/data/First_table.csv")

In [4]:
#Filling orto_row column (sounds like an oxymoron)
assemblies = orto_rows.columns[3:-1]
row_assigner = pd.melt(orto_rows, id_vars=["ortologus_row"], value_vars=assemblies)
row_assigner = row_assigner.query("value != '*'")
row_assigner["value"] = row_assigner["value"].str[:14]
row_assigner
row_assigner.drop(["variable"], axis="columns", inplace=True)
row_assigner.columns = ["ortologus_row", "id"]
df1 = df1.merge(row_assigner, on="id")

In [5]:
# Number of the paralogs
print("Number of the paralogs =", sum(orto_rows.query("Genes > Species").Genes) - sum(orto_rows.query("Genes > Species").Species))

Number of the paralogs = 65


In [6]:
# Creating a subset without pararows
pararows_numbers = orto_rows.query("Genes > Species").ortologus_row
df1 = df1.query("ortologus_row not in @pararows_numbers").query("ortologus_row != 0")

In [7]:
# Compiling data about start-codons of ortologus rows
start_codon_per_row = df1.groupby("ortologus_row", as_index=False).agg({"start_codone": ".".join})
start_codon_per_row["start_codone"]
orr_start_list = []
for number in tqdm(range(len(start_codon_per_row))):
    orr_start_list.append(start_codon_per_row.iloc[number, 1].split("."))
start_codons = pd.DataFrame(
    {
        "ortologus_row": start_codon_per_row["ortologus_row"],
        "start_codons": pd.Series(orr_start_list),
        "ATG": 0.0,
        "GTG": 0.0,
        "TTG": 0.0,
    }
)

# Computing frequencies of exact start-codons
start_codons.sort_values("ortologus_row", inplace=True)
for row in tqdm(range(len(start_codons))):
    freqs = pd.Series(start_codons["start_codons"][row]).value_counts() # If it will be need to visualize percents, use normalize=True in brackets
    rowlength = len(start_codons["start_codons"][row])
    if "ATG" in freqs:
        start_codons["ATG"][row] = freqs["ATG"]
    else:
        start_codons["ATG"][row] = 0
    if "GTG" in freqs:
        start_codons["GTG"][row] = freqs["GTG"]
    else:
        start_codons["GTG"][row] = 0
    if "TTG" in freqs:
        start_codons["TTG"][row] = freqs["TTG"]
    else:
        start_codons["TTG"][row] = 0

  0%|                                                                                                               | 0/1525 [00:00<?, ?it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 10%|█████████▍                                                                                         | 145/1525 [00:00<00:00, 1442.71it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 19%|██████████████████▊                                                                                | 290/1525 [00:00<00:00, 1394.31it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 28%|███████████████████████████▉                                                                       | 430/1525 [00:00<00:00, 1242.48it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 38%|█████████████████████████████████████▎                                                             | 574/1525 [00:00<00:00, 1312.60it/s]/tmp/ipykernel_40437/1259367515.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = freqs["GTG"]
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

...

 65%|████████████████████████████████████████████████████████████████▏                                  | 989/1525 [00:00<00:00, 1353.53it/s]/tmp/ipykernel_40437/1259367515.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = freqs["GTG"]
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 74%|████████████████████████████████████████████████████████████████████████▎                         | 1125/1525 [00:00<00:00, 1341.78it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 83%|█████████████████████████████████████████████████████████████████████████████████▍                | 1267/1525 [00:00<00:00, 1364.61it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

 93%|██████████████████████████████████████████████████████████████████████████████████████████▉       | 1415/1525 [00:01<00:00, 1397.43it/s]/tmp/ipykernel_40437/1259367515.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["ATG"][row] = freqs["ATG"]
/tmp/ipykernel_40437/1259367515.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_codons["GTG"][row] = 0
/tmp/ipykernel_40437/1259367515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 1525/1525 [00:01<00:00, 1356.51it/s]


In [8]:
# Computing uniformity of start-codon per ortologus row
start_codons["uniformity"] = "NA"
for row in range(len(start_codons)):
    if len(set(start_codons.iloc[row, 1])) == 1:
        start_codons.iloc[row, 5] = "same"
    else:
        start_codons.iloc[row, 5] = "different"

In [9]:
# Constructing table withh all data about the ortologus rows (including pararows)
start_codons2 = start_codons.merge(orto_rows, on="ortologus_row", how="outer")

In [10]:
# Creating a table, combining data about gene and its start-codone
strain_gene_row = start_codons2[["Species", "Genes", "ortologus_row", "uniformity", "ATG", "GTG", "TTG"]]   # Other codons deleted
summary_rows = df1.merge(strain_gene_row, on="ortologus_row")

In [11]:
# Assigning cog to orto_row
row_cog = row_assigner.merge(df1[["id", "cog"]], on="id").groupby("ortologus_row").agg({"cog": "max"})
start_codons2 = start_codons2.merge(row_cog, on="ortologus_row")

In [12]:
# Adding organism_name to datasets
summary_rows["organism"] = folder_name
start_codons2["organism"] = folder_name

In [24]:
uniq_cog = start_codons2[["ortologus_row", "ATG", "GTG", "TTG", "uniformity", "Species", "Genes", "cog", "organism"]]

,ortologus_row,ATG,GTG,TTG,uniformity,Species,Genes,cog,organism
0,1,27.0,0.0,0.0,same,27,27,absent,C_psittaci
1,2,27.0,0.0,0.0,same,27,27,COG0448,C_psittaci
2,3,0.0,0.0,27.0,same,27,27,absent,C_psittaci
3,4,1.0,0.0,26.0,different,27,27,absent,C_psittaci
4,5,27.0,0.0,0.0,same,27,27,absent,C_psittaci
...,...,...,...,...,...,...,...,...,...
1520,1528,1.0,0.0,0.0,same,1,1,absent,C_psittaci
1521,1529,1.0,0.0,0.0,same,1,1,absent,C_psittaci
1522,1530,1.0,0.0,0.0,same,1,1,absent,C_psittaci
1523,1531,1.0,0.0,0.0,same,1,1,absent,C_psittaci


We need to align only rows with different start-codons.

In [13]:
# non_uniform_or_list = list(set(start_codons2.query("uniformity == 'different'").ortologus_row))
# for orto_row in tqdm(non_uniform_or_list):
#     subset = summary_rows.query("ortologus_row == @orto_row")
#     with open (f"../{folder_name}/data/multialignments/{folder_name}_withoutp_{str(orto_row)}.fasta", "w") as nucleotide_fasta:
#         for index, row in subset.iterrows():
#             #if row['type_of_the_gene'] != "pseudogene": # It will be actual when we found pseudogenes again
#             nucleotide_fasta.write(">")
#             nucleotide_fasta.write(row["id"])
#             nucleotide_fasta.write("_")
#             nucleotide_fasta.write(row["source"])
#             nucleotide_fasta.write("_")
#             nucleotide_fasta.write(row["start_codone"])
#             nucleotide_fasta.write("\n")
#             nucleotide_fasta.write(row["n_sequence"])
#             nucleotide_fasta.write("\n")

#### Saving summary-rows and start-codons2

In [14]:
#summary_rows.to_csv(f"../{folder_name}/data/summary_rows_prokka.csv", index=False)

In [15]:
#start_codons2.to_csv(f"../{folder_name}/data/start_codons2_prokka.csv", index=False)

In [ ]:
#uniq_cog.to_csv(f"../{folder_name}/data/uniq_cog.csv", index=False)